In [11]:
import numpy as np
import pytest
from numpy.typing import NDArray
from ska_sdp_datamodels.visibility import Visibility as RASCILVisibility
from typing_extensions import assert_never
import os
import pickle
from karabo.imaging.imager_base import DirtyImagerConfig
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging.imager_rascil import RascilDirtyImager
from karabo.imaging.util import (
    auto_choose_dirty_imager_from_sim,
    auto_choose_dirty_imager_from_vis,
)
from karabo.imaging import imager_wsclean
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Tkagg')
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    return

#telescope_path = '/home/rohit/skao_repo/ska1mid.tm'
#telescope_path = '/home/rohit/skao_repo/ska1low.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AA0.5.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AA0.5.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AA1.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AA1.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AA2.tm'
#telescope_path = '/home/rohit/skao_repo/ska-mid-AA2.tm'
#telescope_path = '/home/rohit/skao_repo/ska-low-AAstar.tm'
telescope_path = '/home/rohit/skao_repo/ska-mid-AAstar.tm'

# Code Below does not have correct latitute and longitude 
"""     telescope=Telescope.read_OSKAR_tm_file(telescope_path)
    telescope.read_OSKAR_tm_file(telescope_path)
    telescope.plot_telescope()
    #plt.xlim(20.4,22.0)
    #plt.ylim(-29.9,-31.7)
    plt.title('SKA-mid AA0.5')
    plt.savefig('/media/rohit/data/Dropbox/SKA_solar/manuscript/'+telescope_path.split('/')[-1]+'.png',dpi=100)
    plt.show() """

telescope=Telescope.read_OSKAR_tm_file(telescope_path)
xy=np.loadtxt(telescope.path+'/layout.txt')/1000.
print(xy.shape)
enux,enuy=xy[:,0],xy[:,1]
print(enux,enuy)
f,ax=plt.subplots(1,1)
ax.scatter(enux,enuy,color='red',marker='o',s=50)
ax.scatter(0,0,color='k',s=100,marker='+')
ax.set_xlim(-100.e0,100.e0)
ax.set_ylim(-100.e0,100.e0)
ax.set_xlabel('ENU-X (km)')
ax.set_ylabel('ENU-Y (km)')
ax.set_title('SKA-MID AA* | Total Elements:'+str(int(enux.shape[0])))
plt.savefig('/media/rohit/data/Dropbox/SKA_solar/manuscript/'+telescope_path.split('/')[-1]+'.png',dpi=100)
plt.close()


In [2]:
#telescope_path = '/media/rohit/sdata/ska-solar-files/meerkat.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1mid.tm'
telescope_path = '/home/rohit/skao_repo/ska-mid-AAstar.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/mwa.phase1.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1low.tm'
telescope_name='skamid_AAstar_snap0_100MHz-2-2000GHz'
hour_=0
minutes_=1
noise_enable_ = False
enable_array_beam=False
point_source_bool = False # tag 1 / tag 0 is ideal case
random_points = False # tag 2
nchan = 1
ntchan = 1
if(random_points==True):
    size_ = 100
else:
    size_ = 1
point_flux = 100.e4

In [3]:
layout=np.loadtxt(telescope_path+'/layout.txt')
nant=len(layout)
nb=int(nant*(nant-1)*0.5)
print('Number of Baselines:',nb)
base_length=[0]*nb
k=0
for i in range(nant):
    for j in range(i,nant):
        if(i!=j):
            base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
            k=k+1
base_length=np.array(base_length)
print('Maximum Baseline',base_length.max())
#----------------------------
freq_list = np.arange(1,20)*100 # in MHz
beamsize_arr = 3.e8/base_length.max()/(freq_list*1.e6)
beamsize_arr_arcsec = beamsize_arr*180/np.pi*3600
f,ax=plt.subplots(1,1)
ax.plot(freq_list,beamsize_arr_arcsec,'o-')
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Max. Resolution (arcsec)')
ax.set_title('SKA-mid')
plt.close()

Number of Baselines: 10296
Maximum Baseline 106550.3934796015


**Run Simulation**

In [95]:
for i in range(18):
    start_frequency_hz_ = freq_list[i]*1.e6
    beam_size_arcsec = 3.e8/start_frequency_hz_/base_length.max()*180/np.pi*3600
    print('Maximum Baseline',base_length.max(), 'Beam (arcsec)',beam_size_arcsec)
    print("Frequency (MHz): ",start_frequency_hz_)
    path = '/media/rohit/sdata/ska-solar-files/sim1/'
    aa=fits.open('/media/rohit/sdata/ska-solar-files/sim/20151203_240MHz_psimas.fits')
    ms_filename = path+"solar_"+"freq_"+str(int(freq_list[i]))+'_'+telescope_name+".ms"
    vis_filename = path+"solar_"+"freq_"+str(int(freq_list[i]))+'_'+telescope_name+".vis"
    solar_map=aa[0].data;solar_map_jy=solar_map/np.nanmax(solar_map)*20*1.e4*(start_frequency_hz_/2.4e8)**1
    ra_sun_center=249.141666667;dec_sun_center=21.986 #16 34 34.52 -21 59 09.7
    ra_grid,dec_grid=np.meshgrid((np.arange(256)-128)*22.5/3600.,(np.arange(256)-128)*22.5/3600.)
    ra_grid=ra_grid+ra_sun_center;dec_grid=dec_grid+dec_sun_center
    idx=np.where(solar_map>0.1*np.nanmax(solar_map))
    sky_model_ra=ra_grid[idx];sky_model_dec=dec_grid[idx];flux=solar_map_jy[idx]
    # Simulation starts 
    sky = SkyModel()
    sky_data = np.array([sky_model_ra, sky_model_dec, flux,np.zeros(len(flux)), \
        np.zeros(len(flux)),np.zeros(len(flux)), np.zeros(len(flux)),np.zeros(len(flux)), \
    np.zeros(len(flux)),np.zeros(len(flux)), np.zeros(len(flux)),np.zeros(len(flux))]).T
    sky_data=sky_data[0:16000,:]
    # Add Point Soure
    add_source = np.zeros((size_,12))
    if(point_source_bool):
        add_source[0][0] = ra_sun_center # RA
        add_source[0][1] = dec_sun_center # DEC
        add_source[0][2] = point_flux # Flux
        sky_data1 = np.vstack((sky_data,add_source))
    elif(random_points):
        ra_point_array = np.random.uniform(low=ra_sun_center-0.3, high=ra_sun_center+0.3, size=size_)
        dec_point_array = np.random.uniform(low=dec_sun_center-0.3, high=dec_sun_center+0.3, size=size_)
        flux_array = np.random.uniform(low=1.e1, high=1.e2, size=size_)
        add_source[:,0] = ra_point_array # RA
        add_source[:,1] = dec_point_array # DEC
        add_source[:,2] = flux_array # Flux  
        sky_data1 = np.vstack((sky_data,add_source))
    else:
        sky_data1 = sky_data
    print('Maximum flux density (SFU):',sky_data1[:,2].max()/1.e4)
    print('Number of Sources:',len(sky_data1))
    sky.add_point_sources(sky_data1)
    np.savetxt(path+"source_"+telescope_name+".txt", add_source.T)
    #telescope_name="SKA1MID"
    #telescope=Telescope.get_SKA1_LOW_Telescope()
    dtime=datetime(2000, 1, 1, 10, 0, 00, 0) # MeerKAT/ SKA-mid
    #dtime=datetime(2000, 1, 1, 4, 0, 00, 0) # MWA / SKA-low
    backend=SimulatorBackend.OSKAR
    #telescope = Telescope.constructor(telescope_name, backend=backend)
    telescope=Telescope.read_OSKAR_tm_file(telescope_path)
    telescope.read_OSKAR_tm_file(telescope_path)
    simulation = InterferometerSimulation(vis_path=vis_filename, ms_file_path=ms_filename,
        channel_bandwidth_hz=1, time_average_sec=10, noise_enable=noise_enable_, use_gpus=True,
        noise_seed="time", noise_freq="Range", noise_rms="Range", 
        noise_start_freq=1.e9, noise_inc_freq=1.e8, noise_number_freq=24, 
        noise_rms_start=0, noise_rms_end=0, enable_numerical_beam=enable_array_beam,
        enable_array_beam=enable_array_beam) 
    observation = Observation(mode='Tracking',phase_centre_ra_deg=ra_sun_center, start_date_and_time=dtime, 
        length=timedelta(hours=hour_, minutes=minutes_, seconds=0, milliseconds=0), 
        phase_centre_dec_deg=dec_sun_center, number_of_time_steps=ntchan, 
        start_frequency_hz=start_frequency_hz_, frequency_increment_hz=1, 
        number_of_channels=nchan, ) 
    visibility = simulation.run_simulation(telescope, sky, observation, backend=backend)
    npix_per_beam = 3
    imgsize=4096
    cellsize_arcsec=1 #beam_size_arcsec/npix_per_beam
    cellsize_rad=cellsize_arcsec/3600*np.pi/180 # in rad
    print('Cellsize:',cellsize_arcsec,'Beam (arcsec):',beam_size_arcsec,'Max length:',base_length.max())
    print('Field of View (deg):',imgsize*cellsize_arcsec/3600.)
    dirty_imager = OskarDirtyImager(
        OskarDirtyImagerConfig(
            imaging_npixel=imgsize,
            imaging_cellsize=cellsize_rad,
        ))
    dirty_oskar_img = path+"solar_"+"freq_"+str(int(freq_list[i]))+'_'+telescope_name+"_oskar.fits"
    dirty_image = dirty_imager.create_dirty_image(visibility,output_fits_path=dirty_oskar_img)
    dirty_wsclean_img = path+"solar_"+"freq_"+str(int(freq_list[i]))+'_'+telescope_name+"_wsclean.fits"
    img_cmd = 'wsclean \
            -size '+str(imgsize)+' '+str(imgsize)+' \
            -name '+dirty_wsclean_img+' \
            -scale '+str(cellsize_rad)+'rad -niter 25000 -mgain 0.8 \
            -weight uniform\
            -maxuv-l 9000 -minuv-l 10\
            -channels-out '+str(nchan)+' '+ms_filename
    print(img_cmd)
    try:
        restored = imager_wsclean.create_image_custom_command(command=img_cmd)
    except:
        pass # doing nothing on exception    
    

Maximum Baseline 106550.3934796015 Beam (arcsec) 5.807528236484213
Frequency (MHz):  100000000.0
Maximum flux density (SFU): 8.333333333333334
Number of Sources: 8063
Saved visibility to /media/rohit/sdata/ska-solar-files/sim1/solar_freq_100_skamid_AAstar_snap0_100MHz-2-2000GHz.vis
Cellsize: 1 Beam (arcsec): 5.807528236484213 Max length: 106550.3934796015
Field of View (deg): 1.1377777777777778


/home/rohit/karabo/Karabo-Pipeline/karabo/imaging/image.py:118: UserWarning: Received 3D data for image object.
                Will assume the 3 axes correspond to
                (polarisations, pixels_x, pixels_y).
                Inserting 1 additional axis for frequencies.
  warnings.warn(


wsclean             -size 4096 4096             -name /media/rohit/sdata/ska-solar-files/sim1/solar_freq_100_skamid_AAstar_snap0_100MHz-2-2000GHz_wsclean.fits             -scale 4.84813681109536e-06rad -niter 25000 -mgain 0.8             -weight uniform            -maxuv-l 9000 -minuv-l 10            -channels-out 1 /media/rohit/sdata/ska-solar-files/sim1/solar_freq_100_skamid_AAstar_snap0_100MHz-2-2000GHz.ms
Creating /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-JeOCrkbk8p for Disk cache for WSClean custom command images
WSClean command: [cd /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-JeOCrkbk8p && OPENBLAS_NUM_THREADS=1 wsclean             -size 4096 4096             -name /media/rohit/sdata/ska-solar-files/sim1/solar_freq_100_skamid_AAstar_snap0_100MHz-2-2000GHz_wsclean.fits             -scale 4.84813681109536e-06rad -niter 25000 -mgain 0.8             -weight uniform            -maxuv-l 9000 -minuv-l 10            -channels-out 1 /media/rohit/sdata/ska-solar-files/sim1/solar_

In [19]:
fname_img=dirty_wsclean_img+'-image.fits'
fname_model=dirty_wsclean_img+'-model.fits'
fname_residual=dirty_wsclean_img+'-residual.fits'
fname_psf=dirty_wsclean_img+'-psf.fits'
fname_dirty=dirty_wsclean_img+'-dirty.fits'
d_mod,h=fits.getdata(fname_model,header=True)
d_res,h=fits.getdata(fname_residual,header=True)
d_psf,h=fits.getdata(fname_psf,header=True)
d_dir,h=fits.getdata(fname_dirty,header=True)
d_img,h=fits.getdata(fname_img,header=True)
d_comb = np.mean(np.vstack((d_mod,d_res,d_psf,d_dir,d_img)),axis=1)
print(d_comb.shape)
pickle.dump([h,d_comb],open(dirty_wsclean_img+'.p','wb'))
os.system('rm -rf '+dirty_wsclean_img+'*.fits')

(5, 4096, 4096)


0

**Make an Image**

In [5]:
# IMAGING WSCLEAN

dirty_wsclean_img = path+"solar_"+"freq_"+str(int(freq_list[i]))+'_'+telescope_name+"_wsclean.fits"
img_cmd = 'wsclean \
        -size '+str(imgsize)+' '+str(imgsize)+' \
        -name '+dirty_wsclean_img+' \
        -scale '+str(cellsize_rad)+'rad -niter 1 -mgain 0.8 \
        -weight uniform\
        -maxuv-l 9000 -minuv-l 10\
        -channels-out '+str(nchan)+' '+ms_filename
print(img_cmd)
restored = imager_wsclean.create_image_custom_command(command=img_cmd)

wsclean         -size 4096 4096         -name /media/rohit/sdata/ska-solar-files/sim1/solar_freq_200_skamid_AAstar_snap0_100MHz-2-2000GHz_wsclean.fits         -scale 4.84813681109536e-06rad -niter 1 -mgain 0.8         -weight uniform        -maxuv-l 9000 -minuv-l 10        -channels-out 1 /media/rohit/sdata/ska-solar-files/sim1/solar_freq_200_skamid_AAstar_snap0_100MHz-2-2000GHz.ms
Creating /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-7EZLbgYdiS for Disk cache for WSClean custom command images
WSClean command: [cd /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-7EZLbgYdiS && OPENBLAS_NUM_THREADS=1 wsclean         -size 4096 4096         -name /media/rohit/sdata/ska-solar-files/sim1/solar_freq_200_skamid_AAstar_snap0_100MHz-2-2000GHz_wsclean.fits         -scale 4.84813681109536e-06rad -niter 1 -mgain 0.8         -weight uniform        -maxuv-l 9000 -minuv-l 10        -channels-out 1 /media/rohit/sdata/ska-solar-files/sim1/solar_freq_200_skamid_AAstar_snap0_100MHz-2-2000GHz.ms]
WSClea

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-7EZLbgYdiS/wsclean-image.fits'

**Read Visibilities**

In [6]:
from oskar.measurement_set import MeasurementSet
res_read=MeasurementSet.open(ms_filename,readonly=True)
num_baselines_=int(res_read.num_stations*(res_read.num_stations-1)/2.)
uu,vv,ww=res_read.read_coords(start_row=0,num_baselines=num_baselines_)
vis=res_read.read_vis(start_row=0,start_channel=0,num_channels=1,num_baselines=num_baselines_)
print(vis.shape,uu)
uul, vvl, wwl = uu/3.e8*start_frequency_hz_, vv/3.e8*start_frequency_hz_, ww/3.e8*start_frequency_hz_
#----------------------


(1, 19306, 4) [ 1.56640625e+01 -2.53183594e+01 -5.27451172e+01 ...  5.96445312e+03
  1.91770410e+04  1.32125879e+04]


In [7]:
f,ax=plt.subplots(1,1)
ax.plot(uul/1.e3,vvl/1.e3,'o',c='r',markersize=1)
ax.plot(-uul/1.e3,-vvl/1.e3,'o',c='r',markersize=1)
ax.set_xlabel('U ($k\lambda$)')
ax.set_ylabel('V ($k\lambda$)')
plt.close()
#---------------------
uulc=np.hstack((uul,-uul))/1.e3
vvlc=np.hstack((vvl,-vvl))/1.e3
uvdistlc = np.sqrt(uulc**2 + vvlc**2)
print(uulc.shape)
f,ax=plt.subplots(1,1)
ax.hist(uulc, bins=40,color='k')
ax.set_yscale('log')
ax.set_xlabel('UV Distance ($k\lambda$)')
plt.show()



(38612,)


In [94]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fitsname=ms_filename.split('.m')[0]+"_wsclean.fits-image.fits"
print(fitsname)
d,h=fits.getdata(fitsname,header=True)
d=d[:,:,::-1,::-1][0][0]
l1 = ra_sun_center + h['CDELT1']*h['NAXIS1']/2
l2 = ra_sun_center - h['CDELT1']*h['NAXIS1']/2
r1 = dec_sun_center + h['CDELT2']*h['NAXIS2']/2
r2 = dec_sun_center - h['CDELT2']*h['NAXIS2']/2
print(d.shape,npix_per_beam,r1,r2)
ra_point_array = sky_data1[:,0]
dec_point_array = sky_data1[:,1]
npix_per_beam = beam_size_arcsec/cellsize_arcsec
if(npix_per_beam<1):
    npix_per_beam=1
print(npix_per_beam,(np.pi*npix_per_beam**2)/1.e4)

f,((ax0,ax1),(ax2,ax3))=plt.subplots(2,2)
im0=ax0.imshow(solar_map_jy/1.e4,origin='lower', extent=[ra_grid[0][0],ra_grid[0][-1],dec_grid[:,0][0],dec_grid[:,0][-1]])
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='5%', pad=0.05)
f.colorbar(im0, cax=cax, orientation='vertical',label = 'SFU')
contx=np.linspace(ra_grid[0][0],ra_grid[0][-1],ra_grid[0].shape[0])
conty=np.linspace(dec_grid[:,0][0],dec_grid[:,0][-1],dec_grid[0].shape[0])
ax0.contour(contx,conty,solar_map_jy/np.nanmax(solar_map_jy),levels=[0.5,0.9],colors='white')
ax0.set_xlim(l1,l2)
ax0.set_ylim(r2,r1)
#---------------------------------
#im1=ax1.imshow(d/1.e4,origin='lower', extent = [l1,l2,r1,r2])
im1=ax1.imshow(d/(npix_per_beam**2)/1.e4,origin='lower', extent = [l1,l2,r1,r2])
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
f.colorbar(im1, cax=cax, orientation='vertical',label = 'SFU')
contx=np.linspace(l1,l2,d.shape[0])
conty=np.linspace(r1,r2,d.shape[0])
ax1.contour(contx,conty,d/np.nanmax(d),levels=[0.5,0.9],colors='white')
ax1.set_xlim(l1,l2)
ax1.set_ylim(r2,r1)
#---------------------------------
ax2.plot(ra_point_array,dec_point_array,'o',color='k',markersize=0.5)
c1 = plt.Circle(( ra_sun_center, dec_sun_center ), 0.25, color='red',fill = None )
ax2.add_patch(c1)
ax3.imshow(solar_map_jy,origin='lower', extent=[ra_grid[0][0],ra_grid[0][-1],dec_grid[:,0][0],dec_grid[:,0][-1]])
#ax3.plot(ra_point_array,dec_point_array,'o',color='k')
ax0.set_xlabel('R.A. (Deg)');ax1.set_xlabel('R.A. (Deg)');ax2.set_xlabel('R.A. (Deg)');ax3.set_xlabel('R.A. (Deg)')
ax0.set_ylabel('DEC (deg)');ax1.set_ylabel('DEC (deg)');ax2.set_ylabel('DEC (deg)');ax3.set_ylabel('DEC (deg)')
plt.show()

/media/rohit/sdata/ska-solar-files/sim1/solar_freq_1900_skamid_AAstar_snap0_100MHz-2-2000GHz_wsclean.fits-image.fits
(4096, 4096) 1 22.55488888888889 21.41711111111111
1 0.0003141592653589793


In [47]:
print(h)

SIMPLE  =                    T / file does conform to FITS standard             BITPIX  =                  -32 / number of bits per data pixel                  NAXIS   =                    4 / number of data axes                            NAXIS1  =                 4096 / length of data axis 1                          NAXIS2  =                 4096 / length of data axis 2                          NAXIS3  =                    1 / length of data axis 3                          NAXIS4  =                    1 / length of data axis 4                          EXTEND  =                    T / FITS dataset may contain extensions            COMMENT   FITS (Flexible Image Transport System) format is defined in 'AstronomyCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H BSCALE  =                   1.                                                  BZERO   =                   0.                                                  BUNIT   = 'JY/BEAM '           / Units a

In [ ]:
#telescope.plot_telescope()
#plt.show()

solar_freq_400_skamid_AAstar_snap0_100MHz-2-2000GHz_wsclean.fits-image

In [ ]:
print(npix_per_beam)

0.0982856464058907
